# Is Fandango Still Inflating Ratings

In October 2015, a data journalist named Walt Hickey analyzed movie ratings data and found strong evidence to suggest that Fandango's rating system was biased and dishonest ([Fandango](https://www.fandango.com/) is an online movie ratings aggregator). He published his analysis [in this article](https://fivethirtyeight.com/features/fandango-movies-ratings/).

Fandango's officials said that this was due to a bug in their system, and that they would fix the bug.

In this project, we are going to analyze recent movie ratings data to determine if they fixed the bug since Walt Hickey's analysis.

In [5]:
import pandas as pd
prev = pd.read_csv('fandango_score_comparison.csv')
after = pd.read_csv('movie_ratings_16_17.csv')

fand_prev = score_comparison[[
    'FILM', 'Fandango_Stars', 'Fandango_Ratingvalue', 'Fandango_votes', 'Fandango_Difference'
]].copy()

fand_after = ratings[['movie', 'year', 'fandango']].copy()

fand_after.head(10)

,movie,year,fandango
0,10 Cloverfield Lane,2016,3.5
1,13 Hours,2016,4.5
2,A Cure for Wellness,2016,3.0
3,A Dog's Purpose,2017,4.5
4,A Hologram for the King,2016,3.0
5,A Monster Calls,2016,4.0
6,A Street Cat Named Bob,2016,4.5
7,Alice Through the Looking Glass,2016,4.0
8,Allied,2016,4.0
9,Amateur Night,2016,3.5


# Population of Interest
The population we would like to describe is all of the available ratings that fandango displays on their website, regardless of when the film was released. As stated earlier, our goal is to see if Fandango fixed the "bug" that biased their movie ratings, making their distribution even more left-skewed, inflating their ratings of movies.

Unfortunately, the samples that we have, Hickey's original dataset and our 2016-2017 dataset, are not entirely likely to be representative of our population; our datasets filter out ratings and ticket sales or release date and votes respectively. A later analysis might involve re-sampling the data, though that may be impossible since Fandango "fixed" the availability of some of the data originally sampled in 2015 by Hickey. For now, we'll change the goal of our analysis based on the data available.